# Merging Faces

In [7]:
import cv2
import numpy as np
import dlib
import sys
from time import sleep


# code that specifies our variables
predictor_path = "C:\Users\NEO\Anaconda2\shape_predictor_68_face_landmarks.dat"
scale_factor = 1
feather_amount = 11

# code that assigns a list of range of values based on the number order of the facial points landmark of the predictor.
face_points = list(range(17,68))
mouth_points = list(range(48,61))
right_brow_points = list(range(17,22))
left_brow_points = list(range(22,27))
right_eye_points = list(range(36,42))
left_eye_points = list(range(42,48))
nose_points = list(range(27,35))
jaw_points = list(range(0,17))

# code for the points use to line up the images
allign_points = (left_brow_points + right_eye_points + left_eye_points + right_brow_points + nose_points + mouth_points)


# code that points the image_2 to overlay on the first.
overlay_points = [left_eye_points + right_eye_points + left_brow_points + right_brow_points ,nose_points + mouth_points] 


# code that sets the amount of blur to use during the color correction.
# This is a fraction of the pupillary distance.
color_correct_blur_frac = 0.6

# code that defines our detector object first
detector = dlib.get_frontal_face_detector()
# code that defines our predictor object after defining the detector object.
predictor = dlib.shape_predictor(predictor_path)

# code for the classes that handles exceptions
class Too_many_faces(Exception):
    pass
class No_faces(Exception):
    pass

# function that returns the facial landmarks as (x, y) cords
def get_landmarks(im):
    rects = detector(im,1)
    if len(rects) > 1:
        raise Too_many_faces
    if len(rects) == 0:
        raise No_faces
        
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])
    

# function that overlays the landmark points on the image and returns the image
def annonate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0,0], point[0,1])
        cv2.putText(im. str(idx), pos, fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=0.4, color =(255,0,0))
        cv2.circle(im, pos,3, color =(0,255,255))
    return im


def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color = color)
    
    
# function that generates a mask for the image supplied to the function
def get_face_mask(im, landmarks):
    im = np.zeros(im.shape[:2], dtype=np.float64)
    
    for group in overlay_points:
        draw_convex_hull(im, landmarks[group], color=1)
        
    im = np.array([im,im,im]).transpose((1,2, 0))
    
    im = (cv2.GaussianBlur(im, (feather_amount, feather_amount), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (feather_amount, feather_amount), 0)
    
    return im


# function that produces the transformation matrix that maps the landmarks from one image to the point.
def transformation_from_points(points_1, points_2):
     
    '''
    # This code solves the procrustes method problem by subtracting the centroids gotten from using the '.mean' function.
    # Then we scale the image by using the standard deviation, this is done by dividing the points whose centroid is subtratced by the std value.
    # to get the standard deviation, we use the function; ".std"
    '''
    points_1 = points_1.astype(np.float64)
    points_2 = points_2.astype(np.float64)
    
    c1 = np.mean(points_1, axis=0)
    c2 = np.mean(points_2, axis=0)
    points_1 -= c1
    points_2 -= c2
    
    s1 = np.std(points_1)
    s2 = np.std(points_2)
    points_1 /= s1
    points_2 /= s2
    
    U, S, Vt = np.linalg.svd(points_1.T * points_2)
    
    R = (U * Vt).T
    
    # code that returns the tranformation matrix derived using the function
    return np.vstack([np.hstack(((s2/s1) * R, c2.T - (s2/s1) * R * c1.T)), np.matrix([0.,0.,1.])])

# code that reads our image and the landmarks:
def read_im_and_landmarks(image):
    im = image
    im = cv2.resize(im, None, fx=1, fy=1, interpolation = cv2.INTER_LINEAR)
    im = cv2.resize(im, (im.shape[1] * scale_factor, im.shape[0] * scale_factor))
    
    s = get_landmarks(im)
    
    return im, s

# function that warps our image mask(im) according to the matrix "M" and 
# the dshape arg is the shape of our image mask.
def warp_im(im, M, dshape):
    output_im = np.zeros(dshape, dtype = im.dtype)
    cv2.warpAffine(im, M[:2], (dshape[1], dshape[0]), dst = output_im, borderMode = cv2.BORDER_TRANSPARENT, flags= cv2.WARP_INVERSE_MAP)
    # code that returns the output mask that has been warped.
    return output_im


# code that corrects the color skintone and lightnig color between the two images.
# This is done by matching the skintone and lighing conditions of the two images.
def correct_colors(im1, im2, landmarks1):
    blur_amount = color_correct_blur_frac * np.linalg.norm(np.mean(landmarks1[left_eye_points], axis=0), np.mean(landmarks1[right_eye_points], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)
    
    # code that helps us to avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)
    
    return (im2.astype(np.float64) * im1_blur.astype(np.float64)/im2_blur.astype(np.float64))


# function that takes in the two images, gets the landmarks, gets the transformation matrix and after all operations.
# gives us our face swapped image. N.B: it warps the image_2 unto the image_1 image.
def swapp_faces(image_1, image_2):
    # code that gets the landmarks of the images
    im1, landmarks1 = read_im_and_landmarks(image_1)
    im2, landmarks2 = read_im_and_landmarks(image_2)
    
    # code that gets the transformation matrix "M".
    M = transformation_from_points(landmarks1[allign_points], landmarks2[allign_points])
    
    # this code gets the mask of the image_2
    mask = get_face_mask(im2, landmarks2)
    #code that gets the warped face mask
    warped_mask = warp_im(mask, M, im1.shape)
    combined_mask = np.max([get_face_mask(im1, landmarks1), warped_mask], axis=0)
    
    warped_im2 = warp_im(im2, M, im1.shape)
    # code that gets the color corrected mask image
    warped_corrected_im2 = correct_colors(im1, warped_im2, landmarks1)
    
    output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask
    cv2.imwrite('Output.jpg', output_im)
    image = cv2.imread('Output.jpg')
    return image


#code that enters the path to the input images used for the face swapper program.
image_1 = cv2.imread('C:\image_for_computer_vision\Hillary.jpg')
image_2 = cv2.imread('C:\image_for_computer_vision\Trump.jpg')

swapped = swapp_faces(image_1, image_2)
cv2.imshow('Face Swapper', swapped)

cv2.waitKey()
cv2.destroyAllWindows()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()